In [1]:
import os

In [2]:
%pwd

'/home/aman/Desktop/ScribeSense/research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    data_path: Path
    model: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from ScribeSense.constants import *
from ScribeSense.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_config(self) -> ModelTrainingConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        create_directories([config.root_dir])
    
        model_trainer_config= ModelTrainingConfig(

            root_dir= config.root_dir,
            data_path= config.data_path,
            model= config.model,
            num_train_epochs= params.num_train_epochs,
            warmup_steps= params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            weight_decay= params.weight_decay,
            logging_steps= params.logging_steps,
            evaluation_strategy= params.evaluation_strategy,
            eval_steps= params.eval_steps,
            save_steps= params.save_steps,
            gradient_accumulation_steps= params.gradient_accumulation_steps
        )

        return model_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, AutoTokenizer

from datasets import load_dataset, load_from_disk
import torch

/home/aman/Desktop/ScribeSense/Scribe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-09-06 19:05:36,903: INFO: config: PyTorch version 2.4.0 available.]


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        

        dataset_pt = load_from_disk(self.config.data_path)
        
        trainer_args = TrainingArguments(
                output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
                per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
                weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
                evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
                gradient_accumulation_steps=self.config.gradient_accumulation_steps
         ) 
        #trainer_args = TrainingArguments(
           # output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            #per_device_train_batch_size=1, per_device_eval_batch_size=1,
            #weight_decay=0.01, logging_steps=10,
            #evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
           # gradient_accumulation_steps=16
        #) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_pt["train"], 
                  eval_dataset=dataset_pt["validation"])
        
        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))

        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)

    model_path = os.path.join("artifacts", "model_trainer", "pegasus-samsum-model", "model.safetensors")  # or 'pytorch_model.bin'

    if os.path.exists(model_path):
        print(f"Model already exists at {model_path}. Skipping training...")
    else:
        print("Model not found. Starting training...")
        model_trainer_config.train()
except Exception as e:
    raise e

[2024-09-06 19:06:07,177: INFO: common: yaml file: config/config.yaml loaded successfully]


[2024-09-06 19:06:07,438: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-06 19:06:07,442: INFO: common: created directory at: artifacts]
[2024-09-06 19:06:07,447: INFO: common: created directory at: artifacts/model_trainer]
Model already exists at artifacts/model_trainer/pegasus-samsum-model/model.safetensors. Skipping training...
